<a href="https://colab.research.google.com/github/saeidsaadatigero/ChatBot-NLP/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 19.2 MB/s eta 0:00:00


In [ ]:
pip install gTTS

In [ ]:
pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=40a61f9e7793cdcafb5b4cce92b0c410fe604da09f7c1a1d34f0c62afcf77f2d
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [ ]:
!apt-get install -y portaudio19-dev
!pip install --upgrade pip setuptools wheel
!pip install PyAudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import torch
import pyaudio
from gtts import gTTS
import playsound
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import speech_recognition as sr

In [ ]:


def list_audio_devices():
    p = pyaudio.PyAudio()
    device_count = p.get_device_count()
    devices = []
    for i in range(device_count):
        info = p.get_device_info_by_index(i)
        devices.append(info)
        print(f"Device {i}: {info['name']} - {info['maxInputChannels']} input channels")
    return devices

In [ ]:
model_name = 'facebook/opt-1.3b'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def recognize_speech():
  recognizer = sr.Recognizer()
  with sr.Microphone() as source:
    print("Hello, I'm MagicaAI. How can I help you?")
    audio = recognizer.listen(source)
    try:
      text = recognizer.recognize_google(audio)
      print(f"You said: {text}")
      return text
    except sr.UnknownValueError:
      print("Ohh sorry, I did't understand what you said.")
    except sr.Requesterror:
      print("500")

In [ ]:
def speak_text(text):
  tts = gTTS(text=text, lang='en')
  filename = 'response.mp3'
  tts.save(filename)
  playsound.playsound(filename)
  os.remove(filename)
# در پروژه های تجاری برای اشغال نشدن حافطه ویس تولید شده رو پاک میکنیم
# os.remove(filename)

In [ ]:
def get_chatbot_response(user_input):
    new_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

In [ ]:
def main():
    p = pyaudio.PyAudio()
    devices = list_audio_devices()
    if not devices:
        print("No audio devices available.")
        return
    device_index = 11
    try:
        device_info = p.get_device_info_by_index(device_index)
        print(f"Using device {device_index}: {device_info['name']}")
    except IOError as e:
        print(f"Error accessing device {device_index}: {e}")
        return

    chat_history_ids = None
    while True:
        text = recognize_speech()
        if text:
            response, chat_history_ids = get_chatbot_response(text, chat_history_ids)
            print(f"Bot: {response}")
            speak_text(response)
            if "GoodBye!" in text.lower():
                break

if __name__ == "__main__":
    main()

No audio devices available.
